# Tutorial 7: Classification (Part II)

### Lecture and Tutorial Learning Goals:

After completing this week's lecture and tutorial work, you will be able to:

* Describe what a test data set is and how it is used in classification.
* Using Python, evaluate classification accuracy using a test data set and appropriate metrics.
* Using Python, execute cross validation in Python to choose the number of neighbours.
* Identify when it is necessary to scale variables before classification and do this using Python
* In a dataset with > 2 attributes, perform k-nearest neighbour classification in Python using the `scikit-learn` package to predict the class of a test dataset.
* Describe advantages and disadvantages of the k-nearest neighbour classification algorithm.


## Handwritten Digit Classification using Python

<img src="https://media.giphy.com/media/UwrdbvJz1CNck/giphy.gif" width = "600"/>

Source: https://media.giphy.com/media/UwrdbvJz1CNck/giphy.gif

MNIST is a computer vision dataset that consists of images of handwritten digits like these:

![](img/MNIST.png)

It also includes labels for each image, telling us which digit it is. For example, the labels for the above images are 5, 0, 4, and 1.


In this tutorial, we’re going to train a classifier to look at images and predict what digits they are. Our goal isn’t to train a really elaborate model that achieves state-of-the-art performance, but rather to dip a toe into using classification with pixelated images. As such, we’re going to keep working with the simple K-nearest neighbour classifier we have been exploring in the last two weeks.



### Using image data for classification

As mentioned earlier, every MNIST data point has two parts: an image of a handwritten digit and a corresponding label. Both the training set and test set contain images and their corresponding labels.

Each image is 28 pixels by 28 pixels. We can interpret this as a big matrix of numbers:

<img src="img/MNIST-Matrix.png" width = "500"/>


We can flatten this matrix into a vector of 28x28 = 784 numbers and give it a class label (here 1 for the number one). It doesn’t matter how we flatten the array, as long as we’re consistent between images. From this perspective, the MNIST images are just a bunch of points in a 784-dimensional vector space, with a very rich structure.

<img src="img/matrix_to_row.png" width = "1000"/>


We do this for every image of the digits we have, and we create a data table like the one shown below that we can use for classification. Note, like any other classification problem that we have seen before, we need many observations for each class. This problem is also a bit different from the first classification problem we have encountered (Wisonsin breast cancer data set), in that we have more than two classes (here we have 10 classes, one for each digit from 0 to 9).

<img src="img/data_table.png" width = "700"/>

This information is taken from: https://tensorflow.rstudio.com/tensorflow/articles/tutorial_mnist_beginners.html

In [ ]:
### Run this cell before continuing.

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier


# functions needed to work with images
# code below sourced from: https://gist.github.com/daviddalpiaz/ae62ae5ccd0bada4b9acd6dbc9008706
# helper function for visualization
def show_digit(arr784):
    plt.imshow(np.array(arr784)[1:].reshape(28, 28), cmap="gray")

**Question 1.0** Multiple Choice:
<br> {points: 1}

How many rows and columns does the array of an image have?

A. 784 columns and 1 row

B. 28 columns and 1 row

C. 18 columns and 18 rows

D. 28 columns and 28 rows 

*Assign your answer to an object called `answer1_0`. Make sure the correct answer is an uppercase letter and to surround your answer with quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_0)).encode("utf-8")+b"8e275bd0c67bc0d4").hexdigest() == "eca8038e11e9f453681185334f2fc2e4874a0fa9", "type of answer1_0 is not str. answer1_0 should be an str"
assert sha1(str(len(answer1_0)).encode("utf-8")+b"8e275bd0c67bc0d4").hexdigest() == "048b44b14d935d5571f2fa83d5a34e5baeb6174d", "length of answer1_0 is not correct"
assert sha1(str(answer1_0.lower()).encode("utf-8")+b"8e275bd0c67bc0d4").hexdigest() == "bb6ba56c39a8bb09b4c1167b58f5502cbb631a58", "value of answer1_0 is not correct"
assert sha1(str(answer1_0).encode("utf-8")+b"8e275bd0c67bc0d4").hexdigest() == "93c4fda851c60748c132e164c25954b38c8d2a77", "correct string value of answer1_0 but incorrect case of letters"

print('Success!')

**Question 1.1** Multiple Choice: 
<br> {points: 1}

Once we linearize the array, how many rows represent a number?

A. 28

B. 784

C. 1

D. 18

*Assign your answer to an object called `answer1_1`. Make sure the correct answer is an uppercase letter and to surround your answer with quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_1)).encode("utf-8")+b"4cd1b682a6f57dc2").hexdigest() == "3049e541b97aef3a6481162c73a8d63d95c9b9a3", "type of answer1_1 is not str. answer1_1 should be an str"
assert sha1(str(len(answer1_1)).encode("utf-8")+b"4cd1b682a6f57dc2").hexdigest() == "b79d61566f2da68c4881761b4e7088f5c335eacb", "length of answer1_1 is not correct"
assert sha1(str(answer1_1.lower()).encode("utf-8")+b"4cd1b682a6f57dc2").hexdigest() == "cb7f1d4fff28dee8b2086fb217c1c106d2b48c22", "value of answer1_1 is not correct"
assert sha1(str(answer1_1).encode("utf-8")+b"4cd1b682a6f57dc2").hexdigest() == "86fe8fdfff1ad7e3bb06cf9558edf785fc4a4450", "correct string value of answer1_1 but incorrect case of letters"

print('Success!')

## 2. Exploring the Data

Before we move on to do the modeling component, it is always required that we take a look at our data and understand the problem and the structure of the data well. We can start this part by loading the images and taking a look at the first rows of the dataset. You can load the data set by running the cell below. 

In [ ]:
# Load images.
# Run this cell.
training_data = pd.read_csv("data/mnist_train_small.csv")
testing_data = pd.read_csv("data/mnist_test_small.csv")

Look at the first 6 rows of `training_data`. What do you notice?

In [ ]:
training_data.head(6)

In [ ]:
training_data.shape

There are no class labels! This data set has already been split into the X's (which you loaded above) and the labels. In addition, there is an extra "X" column which represents the row number (1, 2, 3...). **Keep this in mind for now because we will remove it later on.** Now, let's load the labels.

In [ ]:
# Run this cell.
training_labels = pd.read_csv("data/mnist_train_label_small.csv")["y"]
testing_labels = pd.read_csv("data/mnist_test_label_small.csv")["y"]

Look at the first 6 labels of `training_labels` using the `data_frame.head(6)` function. 

In [ ]:
# Use this cell to view the first 6 labels.
# Run this cell.
training_labels.head(6)

In [ ]:
testing_labels.head(6)

**Question 2.0**
<br> {points: 1}

How many rows does the training data set have? Note, each row is a different number in the postal code system. 

Use `len`. Note, the testing data set should have fewer rows than the training data set.

*Assign your answer to an object called `number_of_rows`. Make sure your answer is a numeric and so it should not be surrounded by quotation marks.*

In [ ]:
# your code here
raise NotImplementedError
number_of_rows

In [ ]:
from hashlib import sha1
assert sha1(str(type(number_of_rows)).encode("utf-8")+b"2c58ac7d45f55521").hexdigest() == "38d573b6d4fb3ace039c2a5cd26f67a311c96395", "type of number_of_rows is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(number_of_rows).encode("utf-8")+b"2c58ac7d45f55521").hexdigest() == "849be00e1f8cdfa14afb25fcad4b44ca75415715", "value of number_of_rows is not correct"

print('Success!')

**Question 2.1**
<br> {points: 1}

For mutli-class classification with k-nn it is important for the classes to have about the same number of observations in each class. For example, if  90% of our training set observationas were labeled as 2's, then k-nn classification predict 2 almost every time and we would get an accuracy score of 90% even though our classifier wasn't really doing a great job. 

Use the `value_counts` function to get the counts for each group and see if the data set is balanced across the classes (has roughly equal numbers of observation for each class). Name the output `counts`. 

In [ ]:
# your code here
raise NotImplementedError
counts

In [ ]:
from hashlib import sha1
assert sha1(str(type(len(counts))).encode("utf-8")+b"e86d67f3e5583da7").hexdigest() == "aab4034d3802a80d0bc7e304f264b75836db7fae", "type of len(counts) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(len(counts)).encode("utf-8")+b"e86d67f3e5583da7").hexdigest() == "0e88b5615b2b4915755860db4d1606efb2f71e3e", "value of len(counts) is not correct"

assert sha1(str(type(sum(counts.values))).encode("utf-8")+b"fba9e10017d50fce").hexdigest() == "934e92fe5aa35c6a20c09c26457c3df9a514367e", "type of sum(counts.values) is not correct"
assert sha1(str(sum(counts.values)).encode("utf-8")+b"fba9e10017d50fce").hexdigest() == "997796d641eaee3348994cb072e2b51a43508a75", "value of sum(counts.values) is not correct"

assert sha1(str(type(pd.DataFrame(counts).reset_index().shape)).encode("utf-8")+b"3d03525cebb7c583").hexdigest() == "6ad57afd8f84459dddf0f20421ec2c8d6224f3c8", "type of pd.DataFrame(counts).reset_index().shape is not tuple. pd.DataFrame(counts).reset_index().shape should be a tuple"
assert sha1(str(len(pd.DataFrame(counts).reset_index().shape)).encode("utf-8")+b"3d03525cebb7c583").hexdigest() == "89f22218967f04e55a99ade7f039aaf1efbc2b19", "length of pd.DataFrame(counts).reset_index().shape is not correct"
assert sha1(str(sorted(map(str, pd.DataFrame(counts).reset_index().shape))).encode("utf-8")+b"3d03525cebb7c583").hexdigest() == "bdd96eb64e7f6bc8be52252f523eb109f628f4cc", "values of pd.DataFrame(counts).reset_index().shape are not correct"
assert sha1(str(pd.DataFrame(counts).reset_index().shape).encode("utf-8")+b"3d03525cebb7c583").hexdigest() == "85eb442ad2c9285a2f7524b7a98506c66f58b429", "order of elements of pd.DataFrame(counts).reset_index().shape is not correct"

assert sha1(str(type(sum(pd.DataFrame(counts).reset_index().index))).encode("utf-8")+b"9b346a686cf05f4a").hexdigest() == "0e19b00f056e04605d41747deb1885cbd21c4652", "type of sum(pd.DataFrame(counts).reset_index().index) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(pd.DataFrame(counts).reset_index().index)).encode("utf-8")+b"9b346a686cf05f4a").hexdigest() == "7d9aaebfdbdaa3225dbedb211aa29f89a43b2a7b", "value of sum(pd.DataFrame(counts).reset_index().index) is not correct"

print('Success!')

**Question 2.2** True or False:
<br> {points: 1}

The classes are not balanced. Some of them are *many times* larger or smaller than others. 

*Assign your answer to an object called `answer2_2`. Make sure your answer is a boolean. i.e. `True` or `False`.*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer2_2)).encode("utf-8")+b"ca7f474287ab1268").hexdigest() == "a12e6c3bb230ecaa76d44e6a2948e9de663bf23f", "type of answer2_2 is not bool. answer2_2 should be a bool"
assert sha1(str(answer2_2).encode("utf-8")+b"ca7f474287ab1268").hexdigest() == "1f2d86215242a77692d582178c463102efdc2768", "boolean value of answer2_2 is not correct"

print('Success!')

To view an image in the notebook, you can use the `show_digit` function (we gave you the code for this function in the first code cell in the notebook, All you have to do to use it is run the cell below). The `show_digit` function takes the row from the dataset whose image you want to produce, which you can obtain using the `iloc` method.

The code we provide below will show you the image for the observation in the 200th row from the training data set. 

In [ ]:
# Run this cell to get the images for the 200th row from the training data set.
show_digit(training_data.iloc[199])

**Question 2.3** 
<br> {points: 3}

Show the image for row 102.

In [ ]:
# your code here
raise NotImplementedError

If you are unsure as to what number the plot is depicting (because the handwriting is messy) you can get the label from the `training_labels`:

In [ ]:
# run this cell to get the training label for the 200th row
training_labels[199]

**Question 2.4** 
<br> {points: 1}

What is the class label for row 102? 

*Assign your answer to an object called `label_102`.*

In [ ]:
# your code here
raise NotImplementedError
label_102

In [ ]:
from hashlib import sha1
assert sha1(str(type(label_102)).encode("utf-8")+b"2930bd7f1e1b5854").hexdigest() == "c9723ecb2dd11608213c38df74b4121764631ee8", "type of label_102 is not correct"
assert sha1(str(label_102).encode("utf-8")+b"2930bd7f1e1b5854").hexdigest() == "6fda01ad34deea1369e6f01a223fe207e8146072", "value of label_102 is not correct"

print('Success!')

## 3. Splitting the Data

**Question 3.1**
<br> {points: 3}

We have already split the data into two datasets, one for training purposes and one for testing purposes. **Is it important to split the data into a training and testing dataset when designing a knn classification model?** If yes, why do we do this? If no, explain why this is not a good idea.

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

## Which $k$ should we use?

As you learned from the worksheet, we can use cross-validation on the training data set to select which $k$ is the most optimal for our data set for k-nn classification. 

**Question 3.2**
<br> {points: 1}

To get all the marks in this question, you will have to:
- Create a parameter grid that includes a set of ranges for the hyperparameter that needs to be tuned
- Perform a 5-fold cross-validation on the training set
- Create a grid search model with your `KNeighborsClassifier` and a parameter grid (covering `n_neighbors` from 2 to 14, both inclusive), and *specify `return_train_score=True` and `n_jobs=-1`.* 
- Fit the grid search model to training set
- Get the `cv_results_` from the fitted model
- Plot the $k$ vs the accuracy
    - Assign this plot to an object called `cross_val_plot`

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(1234)

# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(cross_val_plot is None)).encode("utf-8")+b"6619ab54893644d5").hexdigest() == "7037ad61ac6214641b6b7a9ed0d1e3be5b04a1f3", "type of cross_val_plot is None is not bool. cross_val_plot is None should be a bool"
assert sha1(str(cross_val_plot is None).encode("utf-8")+b"6619ab54893644d5").hexdigest() == "119c7b422e569ff436174d43ef6e8d65291a7483", "boolean value of cross_val_plot is None is not correct"

assert sha1(str(type(cross_val_plot.mark.type)).encode("utf-8")+b"3354e18313506d6d").hexdigest() == "36c5106b7f985c374d3cd68e9dd0e073eabf0bae", "type of cross_val_plot.mark.type is not str. cross_val_plot.mark.type should be an str"
assert sha1(str(len(cross_val_plot.mark.type)).encode("utf-8")+b"3354e18313506d6d").hexdigest() == "9a223bec9a48d2f652513f1fdbdf877cae834cdb", "length of cross_val_plot.mark.type is not correct"
assert sha1(str(cross_val_plot.mark.type.lower()).encode("utf-8")+b"3354e18313506d6d").hexdigest() == "149b7b45fd4bd7d40404494b27797a7be8d37bff", "value of cross_val_plot.mark.type is not correct"
assert sha1(str(cross_val_plot.mark.type).encode("utf-8")+b"3354e18313506d6d").hexdigest() == "149b7b45fd4bd7d40404494b27797a7be8d37bff", "correct string value of cross_val_plot.mark.type but incorrect case of letters"

assert sha1(str(type(cross_val_plot.mark['point'])).encode("utf-8")+b"cb751a0e32369b5e").hexdigest() == "04f7d35f1be1bc49db3af8a43d021180bebffcb8", "type of cross_val_plot.mark['point'] is not bool. cross_val_plot.mark['point'] should be a bool"
assert sha1(str(cross_val_plot.mark['point']).encode("utf-8")+b"cb751a0e32369b5e").hexdigest() == "0b3cbefc2dd0af7d5fcbe8adbfa52a386331fb2a", "boolean value of cross_val_plot.mark['point'] is not correct"

assert sha1(str(type(cross_val_plot.encoding.x.field)).encode("utf-8")+b"fcd0002664247477").hexdigest() == "2862b83edb17bb1ac24162e9d05a0bafe07fe092", "type of cross_val_plot.encoding.x.field is not str. cross_val_plot.encoding.x.field should be an str"
assert sha1(str(len(cross_val_plot.encoding.x.field)).encode("utf-8")+b"fcd0002664247477").hexdigest() == "dcac9241ac9e66b468611974602ddd9adbec5065", "length of cross_val_plot.encoding.x.field is not correct"
assert sha1(str(cross_val_plot.encoding.x.field.lower()).encode("utf-8")+b"fcd0002664247477").hexdigest() == "4bcb8715723e9b98bd1d908ff7556a4ea75ce9d7", "value of cross_val_plot.encoding.x.field is not correct"
assert sha1(str(cross_val_plot.encoding.x.field).encode("utf-8")+b"fcd0002664247477").hexdigest() == "4bcb8715723e9b98bd1d908ff7556a4ea75ce9d7", "correct string value of cross_val_plot.encoding.x.field but incorrect case of letters"

assert sha1(str(type(cross_val_plot.encoding.y.field)).encode("utf-8")+b"f25a3b2dd81480f1").hexdigest() == "fe7c39117efcd0584102ba7f8ae1cd7110cd8fc8", "type of cross_val_plot.encoding.y.field is not str. cross_val_plot.encoding.y.field should be an str"
assert sha1(str(len(cross_val_plot.encoding.y.field)).encode("utf-8")+b"f25a3b2dd81480f1").hexdigest() == "e1c7d9208bbe67d9ab50527f68eb63fad76c4b13", "length of cross_val_plot.encoding.y.field is not correct"
assert sha1(str(cross_val_plot.encoding.y.field.lower()).encode("utf-8")+b"f25a3b2dd81480f1").hexdigest() == "5e49a31c0db205e1fa5ddd11d0b5353210c265d7", "value of cross_val_plot.encoding.y.field is not correct"
assert sha1(str(cross_val_plot.encoding.y.field).encode("utf-8")+b"f25a3b2dd81480f1").hexdigest() == "5e49a31c0db205e1fa5ddd11d0b5353210c265d7", "correct string value of cross_val_plot.encoding.y.field but incorrect case of letters"

assert sha1(str(type(sum(cross_val_plot.data.mean_train_score))).encode("utf-8")+b"b458b2287e7e1e6a").hexdigest() == "381a72c77e2e7cd46fb8aad41edba71743e92bc8", "type of sum(cross_val_plot.data.mean_train_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(cross_val_plot.data.mean_train_score), 2)).encode("utf-8")+b"b458b2287e7e1e6a").hexdigest() == "d3133e2db92f1f5f69c2d2ba161006554b3798ce", "value of sum(cross_val_plot.data.mean_train_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(cross_val_plot.data.param_n_neighbors))).encode("utf-8")+b"bcbdb858d5fdb4f1").hexdigest() == "000cd1382568de9f9a89b329466df904bf4dc73c", "type of sum(cross_val_plot.data.param_n_neighbors) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(cross_val_plot.data.param_n_neighbors)).encode("utf-8")+b"bcbdb858d5fdb4f1").hexdigest() == "f45c92874cdb6493c6f3841885883464697f4f02", "value of sum(cross_val_plot.data.param_n_neighbors) is not correct"

print('Success!')

**Question 3.3**
<br> {points: 3}

Based on the plot from **Question 3.2**, which $k$ would you choose and how can you be sure about your decision? In your answer you should reference why we do cross-validation.

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

## 4. Let's build our model

**Question 4.0**
<br> {points: 3}

Now that we have explored our data, separated the data into training and testing sets (was technically done for you), and applied cross-validation to choose the best $k$, we can build our final model.

First, build your model specification with the best value for $K$. Assign your answer to an object called `mnist_spec`.

Then, pass the model specification and the training data set to the `fit` function. 

*Assign your answer to an object called `mnist_fit`.*

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(9999)

# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(mnist_spec is None)).encode("utf-8")+b"7859424d62b6bc49").hexdigest() == "ef7e8a39b56e2d359e638d6a96370365afbd00b3", "type of mnist_spec is None is not bool. mnist_spec is None should be a bool"
assert sha1(str(mnist_spec is None).encode("utf-8")+b"7859424d62b6bc49").hexdigest() == "00e92f02369417833ece05f8e7a4bb6029dcf626", "boolean value of mnist_spec is None is not correct"

assert sha1(str(type(mnist_fit is None)).encode("utf-8")+b"006c22f95352dbe5").hexdigest() == "93be496d1f536fb752f67f1537e4650f21ea48d2", "type of mnist_fit is None is not bool. mnist_fit is None should be a bool"
assert sha1(str(mnist_fit is None).encode("utf-8")+b"006c22f95352dbe5").hexdigest() == "9010b862921405b2b669596155a936b270fb2447", "boolean value of mnist_fit is None is not correct"

assert sha1(str(type(mnist_fit.n_neighbors)).encode("utf-8")+b"751e92cf46257587").hexdigest() == "f8c490f6d1def9e24d3eed168385b3b318eca1cf", "type of mnist_fit.n_neighbors is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(mnist_fit.n_neighbors).encode("utf-8")+b"751e92cf46257587").hexdigest() == "6ba53c27851a6037b914e4b14965340d3b2765dd", "value of mnist_fit.n_neighbors is not correct"

assert sha1(str(type(mnist_fit.weights)).encode("utf-8")+b"e9ae787dc27cfe8d").hexdigest() == "c679faaaa3b91fa6333011cbf630884033ba9d61", "type of mnist_fit.weights is not str. mnist_fit.weights should be an str"
assert sha1(str(len(mnist_fit.weights)).encode("utf-8")+b"e9ae787dc27cfe8d").hexdigest() == "1d93ea20f8b9a66e2473fd2f0c1964fe43e39c97", "length of mnist_fit.weights is not correct"
assert sha1(str(mnist_fit.weights.lower()).encode("utf-8")+b"e9ae787dc27cfe8d").hexdigest() == "a67e699b46cb7a1162f9e0db1f5f30b65a6e7b57", "value of mnist_fit.weights is not correct"
assert sha1(str(mnist_fit.weights).encode("utf-8")+b"e9ae787dc27cfe8d").hexdigest() == "a67e699b46cb7a1162f9e0db1f5f30b65a6e7b57", "correct string value of mnist_fit.weights but incorrect case of letters"

assert sha1(str(type(type(mnist_fit))).encode("utf-8")+b"3b91cca9009ee3ab").hexdigest() == "354a58ebce2eaff5dd806eee360c0018bc31cf44", "type of type(mnist_fit) is not correct"
assert sha1(str(type(mnist_fit)).encode("utf-8")+b"3b91cca9009ee3ab").hexdigest() == "7d2ca876cb082109c38846365b8e653320de3a00", "value of type(mnist_fit) is not correct"

assert sha1(str(type(mnist_fit.n_samples_fit_)).encode("utf-8")+b"0e35cc61bf0a3e5e").hexdigest() == "a7efcb48fd3020dcaab9ed311aa6c409bbc64297", "type of mnist_fit.n_samples_fit_ is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(mnist_fit.n_samples_fit_).encode("utf-8")+b"0e35cc61bf0a3e5e").hexdigest() == "2bdd232e30246d5c18885c1e1ff10e056efd6211", "value of mnist_fit.n_samples_fit_ is not correct"

print('Success!')

**Question 4.1**
<br> {points: 1}

Use your final model to predict on the test dataset and assign this to an object called `mnist_predictions`. Combine the `testing_data`, `mnist_predictions` and `testing_labels` into one dataframe using `pd.concat` function and assign it to an object called `mnist_df`. Report the accuracy of this prediction, and store this in an object named `mnist_metrics`. Also report the confusion matrix and and store this in an object named `mnist_conf_mat`. 

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(9999)

# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert str(type(mnist_predictions is None)) == "<class 'bool'>", "type of mnist_predictions is None is not bool. mnist_predictions is None should be a bool"
assert str(mnist_predictions is None) == "False", "boolean value of mnist_predictions is None is not correct"

assert str(type(sum(mnist_predictions))) == "<class 'numpy.int64'>", "type of sum(mnist_predictions) is not correct"
assert str(sum(mnist_predictions)) == "2548", "value of sum(mnist_predictions) is not correct"

assert str(type(sum(mnist_df.y))) == "<class 'int'>", "type of sum(mnist_df.y) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert str(sum(mnist_df.y)) == "2638", "value of sum(mnist_df.y) is not correct"

assert str(type(sum(mnist_df.predicted))) == "<class 'int'>", "type of sum(mnist_df.predicted) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert str(sum(mnist_df.predicted)) == "2548", "value of sum(mnist_df.predicted) is not correct"

assert str(type(mnist_conf_mat is None)) == "<class 'bool'>", "type of mnist_conf_mat is None is not bool. mnist_conf_mat is None should be a bool"
assert str(mnist_conf_mat is None) == "False", "boolean value of mnist_conf_mat is None is not correct"

assert str(type(mnist_conf_mat)) == "<class 'numpy.ndarray'>", "type of type(mnist_conf_mat) is not correct"

assert str(type(mnist_conf_mat.sum())) == "<class 'numpy.int64'>", "type of mnist_conf_mat.sum() is not correct"
assert str(mnist_conf_mat.sum()) == "600", "value of mnist_conf_mat.sum() is not correct"

assert str(type(mnist_metrics is None)) == "<class 'bool'>", "type of mnist_metrics is None is not bool. mnist_metrics is None should be a bool"
assert str(mnist_metrics is None) == "False", "boolean value of mnist_metrics is None is not correct"

assert str(type(mnist_metrics)) == "<class 'numpy.float64'>", "type of type(mnist_metrics) is not correct"

assert str(type(mnist_metrics)) == "<class 'numpy.float64'>", "type of mnist_metrics is not correct"
assert str(mnist_metrics) == "0.7666666666666667", "value of mnist_metrics is not correct"

print('Success!')

**Question 4.2**
<br> {points: 3}

For this exercise, print out 3 images and the true labels from the test set that were predicted correctly. 

To approach this exercise, we will first create a data frame that contains the predictions, and the labels from the testing set. We will want to use this data to find cases where the hand written digits were predicted correctly (i.e., any rows where the values in the predicted class label column, `predicted`, match the test set labels, `y`). We will then want to use this information to go back to the original data and extract images that correspond to these correct predictions. 

To keep track of which rows correspond to which images, we will use `reset_index` to reset the index of the dataframe to get the row numbers of the dataframe. It will be useful to keep track of which rows correspond to which images when we do something to the data frame, for example, filter it to find correctly predicted labels. Scaffolding has been provided below for you. *Assign your answer to an object called `mnist_predictions_with_labels`.*

From this data frame, filter for cases of equality between the predictions, `predicted`, and testing set labels, `y`. Essentially, you want to find cases where the predictions match the testing set labels. Sample 3 rows of data using `head`. *Assign your answer to an object called `matching`.* 

Next, we want to extract the row numbers (`index`) from the `matching` data frame to input them into the `show_digit` function. The scaffolding to extract the data from the first row and the associated row number is provided below. *Assign your answers respectively to `matching_1`, `matching_2`, and `matching_3`.*

Finally, utilize the `show_digit` function to print our your images by inputting the sliced testing data set where the predictions match the testing set labels (`matching_1`, `matching_2`, `matching_3`).  

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(1000)

# ___ = mnist_df.reset_index()[
#     ["index", ___, ___]
# ]

# ___ = ___[
#     mnist_predictions_with_labels[___] == mnist_predictions_with_labels[___]
# ].head(___)

# ___ = ___.iloc[___.iloc[0, 0]] # the first row and the first column
# ___ = ___.iloc[___.iloc[__, 0]] # the second row and the first column
# ___ = ___.iloc[___.iloc[__, 0]] # the third row and the first column

# your code here
raise NotImplementedError

In [ ]:
show_digit(matching_1)

In [ ]:
show_digit(matching_2)

In [ ]:
show_digit(matching_3)

In [ ]:
from hashlib import sha1
assert sha1(str(type(mnist_predictions_with_labels is None)).encode("utf-8")+b"ae28e7955fc4b165").hexdigest() == "ec133a0511f322e4409ea3dbda7dce78ed2b9ce9", "type of mnist_predictions_with_labels is None is not bool. mnist_predictions_with_labels is None should be a bool"
assert sha1(str(mnist_predictions_with_labels is None).encode("utf-8")+b"ae28e7955fc4b165").hexdigest() == "39e6fe72e3b66a1d6f96feb38924016b75f9cf88", "boolean value of mnist_predictions_with_labels is None is not correct"

assert sha1(str(type(matching is None)).encode("utf-8")+b"e2510c4e72108b33").hexdigest() == "7f7f2cafa0b129896b9a0391c784b0117ed64ada", "type of matching is None is not bool. matching is None should be a bool"
assert sha1(str(matching is None).encode("utf-8")+b"e2510c4e72108b33").hexdigest() == "f21290edf32cdccb7c564d263bbc58694b9a898c", "boolean value of matching is None is not correct"

assert sha1(str(type(matching_1 is None)).encode("utf-8")+b"8ddff3526918fa4e").hexdigest() == "96e763537304a9ee53d7f196a90b5f431f02f721", "type of matching_1 is None is not bool. matching_1 is None should be a bool"
assert sha1(str(matching_1 is None).encode("utf-8")+b"8ddff3526918fa4e").hexdigest() == "a59acfafb6529902ed61cd6f9bd26efe75f6cc9f", "boolean value of matching_1 is None is not correct"

assert sha1(str(type(matching_2 is None)).encode("utf-8")+b"ba4595a534cbb895").hexdigest() == "9a43ffcd8fbd11672ab581ad3ca2245965061ef4", "type of matching_2 is None is not bool. matching_2 is None should be a bool"
assert sha1(str(matching_2 is None).encode("utf-8")+b"ba4595a534cbb895").hexdigest() == "820ad056de025889f1d165057018684d23e4a077", "boolean value of matching_2 is None is not correct"

assert sha1(str(type(matching_3 is None)).encode("utf-8")+b"b70b0e4284932357").hexdigest() == "ba03945270677d1a6fc94e454d2de6d4f7947f79", "type of matching_3 is None is not bool. matching_3 is None should be a bool"
assert sha1(str(matching_3 is None).encode("utf-8")+b"b70b0e4284932357").hexdigest() == "821bf9d8c53230438a7e51ee39b68d02e39c4ace", "boolean value of matching_3 is None is not correct"

assert sha1(str(type(mnist_predictions_with_labels)).encode("utf-8")+b"d6963eca8c92cbf6").hexdigest() == "536617ff07a5e85393ffe7acdb19823965e8ffea", "type of type(mnist_predictions_with_labels) is not correct"

assert sha1(str(type('index' in mnist_predictions_with_labels.columns)).encode("utf-8")+b"dae6e88173fc2528").hexdigest() == "85878c6d6f4edb509a33fcf6b106a1f16cf6f15b", "type of 'index' in mnist_predictions_with_labels.columns is not bool. 'index' in mnist_predictions_with_labels.columns should be a bool"
assert sha1(str('index' in mnist_predictions_with_labels.columns).encode("utf-8")+b"dae6e88173fc2528").hexdigest() == "e3c93e77d4bb7b82812c7fabd3c97189a81b7656", "boolean value of 'index' in mnist_predictions_with_labels.columns is not correct"

assert sha1(str(type(mnist_predictions_with_labels.shape)).encode("utf-8")+b"bc7a1ef7b64e9553").hexdigest() == "81b44ada66d708e4841cba712e7dcfd45e35e3b0", "type of mnist_predictions_with_labels.shape is not tuple. mnist_predictions_with_labels.shape should be a tuple"
assert sha1(str(len(mnist_predictions_with_labels.shape)).encode("utf-8")+b"bc7a1ef7b64e9553").hexdigest() == "ece49e9b183640304b84bba3e3ba2ab19eab1de8", "length of mnist_predictions_with_labels.shape is not correct"
assert sha1(str(sorted(map(str, mnist_predictions_with_labels.shape))).encode("utf-8")+b"bc7a1ef7b64e9553").hexdigest() == "346bea67cf23448ec03536111dc13df7ba7c97ec", "values of mnist_predictions_with_labels.shape are not correct"
assert sha1(str(mnist_predictions_with_labels.shape).encode("utf-8")+b"bc7a1ef7b64e9553").hexdigest() == "a2378a595946bf948325f580913b0b876aa5eab6", "order of elements of mnist_predictions_with_labels.shape is not correct"

assert sha1(str(type(sum(mnist_predictions_with_labels.predicted))).encode("utf-8")+b"35d4b0ac33d0b536").hexdigest() == "904d35ab8934933ce842d43f184099c3852fb53f", "type of sum(mnist_predictions_with_labels.predicted) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(mnist_predictions_with_labels.predicted)).encode("utf-8")+b"35d4b0ac33d0b536").hexdigest() == "343b4202b7242ffbc176a6f71f25ac16129828d0", "value of sum(mnist_predictions_with_labels.predicted) is not correct"

assert sha1(str(type(sum(mnist_predictions_with_labels.index))).encode("utf-8")+b"18c81dd84e4bda4d").hexdigest() == "bc74d2e387a3ee93ebd7aed47ca173c314f14829", "type of sum(mnist_predictions_with_labels.index) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(mnist_predictions_with_labels.index)).encode("utf-8")+b"18c81dd84e4bda4d").hexdigest() == "32a6f6671eccf94570ac803c75d1b078687a71c2", "value of sum(mnist_predictions_with_labels.index) is not correct"

assert sha1(str(type(matching)).encode("utf-8")+b"a1e68874a4f5ad4d").hexdigest() == "e0beec2ff408ab14d0e9873599e89356bdc522f9", "type of type(matching) is not correct"

assert sha1(str(type(matching.shape)).encode("utf-8")+b"656842569d8ed56d").hexdigest() == "246c4c68b5e77a35bb0a64dbe3b6a7661acfa44c", "type of matching.shape is not tuple. matching.shape should be a tuple"
assert sha1(str(len(matching.shape)).encode("utf-8")+b"656842569d8ed56d").hexdigest() == "fd03f70a3e496564fa2a92066e3a442f11598cfe", "length of matching.shape is not correct"
assert sha1(str(sorted(map(str, matching.shape))).encode("utf-8")+b"656842569d8ed56d").hexdigest() == "90f884013a567347856204c93d65d763c98b755f", "values of matching.shape are not correct"
assert sha1(str(matching.shape).encode("utf-8")+b"656842569d8ed56d").hexdigest() == "5fe86d21cf763ba535c26a6038a72dc9120d8635", "order of elements of matching.shape is not correct"

assert sha1(str(type(sum(matching.predicted))).encode("utf-8")+b"785fbae1d8158b0e").hexdigest() == "1e1228520e464a7534a3cf5dadae45bd4b74de3e", "type of sum(matching.predicted) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(matching.predicted)).encode("utf-8")+b"785fbae1d8158b0e").hexdigest() == "89fa19ba48453064c9f37072b4fe072dc48d7c5b", "value of sum(matching.predicted) is not correct"

assert sha1(str(type(sum(matching.index))).encode("utf-8")+b"7750464899957b37").hexdigest() == "2aff4b673835cc31b09cbf91ed6725da37a8627f", "type of sum(matching.index) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(matching.index)).encode("utf-8")+b"7750464899957b37").hexdigest() == "a2026be336800e231dca507405d1a6008ec8a203", "value of sum(matching.index) is not correct"

assert sha1(str(type(matching_1)).encode("utf-8")+b"de1eecab2363f651").hexdigest() == "a5c3f2c1cf2ba23d7ce1746ceb8364e94a20198a", "type of matching_1 is not correct"
assert sha1(str(matching_1).encode("utf-8")+b"de1eecab2363f651").hexdigest() == "b04312a39f27bdbc25c692c40ddc6050e8d4d20f", "value of matching_1 is not correct"

assert sha1(str(type(matching_2)).encode("utf-8")+b"a719a761e3e2f88f").hexdigest() == "91ee086e647f751069eb484f5e813cd3b745954a", "type of matching_2 is not correct"
assert sha1(str(matching_2).encode("utf-8")+b"a719a761e3e2f88f").hexdigest() == "d0ab63680b903236afdab7004f62abe3e9c3478e", "value of matching_2 is not correct"

assert sha1(str(type(matching_3)).encode("utf-8")+b"61671e2e3cbcf434").hexdigest() == "3babcb1240ec2500ff3b3d67997e0e84efee82f4", "type of matching_3 is not correct"
assert sha1(str(matching_3).encode("utf-8")+b"61671e2e3cbcf434").hexdigest() == "4ad596a3685f7e86369852100e874dee9bb414cc", "value of matching_3 is not correct"

print('Success!')

**Question 4.3**
<br> {points: 1}

Print out 3 images and true labels from the test set that were **NOT** predicted correctly. You can reuse the `mnist_predictions_with_labels` data frame from **Question 4.2**. 

Filter for inequality between the predictions and the labels for the testing set in a data frame called `not_matching`. Afterwards, extract the row number and assign them to `not_matching_1`, `not_matching_2`, and `not_matching_3` respectively. If you need help, refer to the instructions in **Question 4.2**. 

Similar to the previous question, use the `show_digit` function we gave you above to print out the images.

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(3500)

# your code here
raise NotImplementedError

In [ ]:
not_matching

In [ ]:
show_digit(not_matching_1)

In [ ]:
show_digit(not_matching_2)

In [ ]:
show_digit(not_matching_3)

In [ ]:
from hashlib import sha1
assert sha1(str(type(not_matching is None)).encode("utf-8")+b"6ac2ac77ead56aa9").hexdigest() == "11b1390dd907f413878ccb5fe5b965989edbd017", "type of not_matching is None is not bool. not_matching is None should be a bool"
assert sha1(str(not_matching is None).encode("utf-8")+b"6ac2ac77ead56aa9").hexdigest() == "4babcd399cbaf1d40603d24c8f1f7e7ddb42e91d", "boolean value of not_matching is None is not correct"

assert sha1(str(type(not_matching_1 is None)).encode("utf-8")+b"a041aa4041c74c9d").hexdigest() == "cece74ce8b2d8d56ae3fabdd86c9969433f7b7a0", "type of not_matching_1 is None is not bool. not_matching_1 is None should be a bool"
assert sha1(str(not_matching_1 is None).encode("utf-8")+b"a041aa4041c74c9d").hexdigest() == "2fd021ca03badf2349b7caf5ad81cd9d846f3c54", "boolean value of not_matching_1 is None is not correct"

assert sha1(str(type(not_matching_2 is None)).encode("utf-8")+b"4d56acd125515411").hexdigest() == "969db7264d2370cb351abc1f3147946e2f2dd519", "type of not_matching_2 is None is not bool. not_matching_2 is None should be a bool"
assert sha1(str(not_matching_2 is None).encode("utf-8")+b"4d56acd125515411").hexdigest() == "93cd203b0832df0a0dde2a0a5a1539f3d70472f5", "boolean value of not_matching_2 is None is not correct"

assert sha1(str(type(not_matching_3 is None)).encode("utf-8")+b"6e76e99076fd1883").hexdigest() == "b40e1bfede61aa1767f4f3eddb6a17d1c496580a", "type of not_matching_3 is None is not bool. not_matching_3 is None should be a bool"
assert sha1(str(not_matching_3 is None).encode("utf-8")+b"6e76e99076fd1883").hexdigest() == "b0c09cbc4ee185b963818a450e2172cf39a802a0", "boolean value of not_matching_3 is None is not correct"

assert sha1(str(type(not_matching)).encode("utf-8")+b"b1da23b0bc370b93").hexdigest() == "da426dc52d6358ae7e6432d9eac6f839fe795b6d", "type of type(not_matching) is not correct"

assert sha1(str(type(not_matching.shape)).encode("utf-8")+b"7b23b7829544f926").hexdigest() == "83a1da0e269ec8d3a643d7321383830877410b89", "type of not_matching.shape is not tuple. not_matching.shape should be a tuple"
assert sha1(str(len(not_matching.shape)).encode("utf-8")+b"7b23b7829544f926").hexdigest() == "6c31b767aa5b777903c06266ccc5fbbbde03076f", "length of not_matching.shape is not correct"
assert sha1(str(sorted(map(str, not_matching.shape))).encode("utf-8")+b"7b23b7829544f926").hexdigest() == "aafa43df872f6e6e29b5c2278928be0716183728", "values of not_matching.shape are not correct"
assert sha1(str(not_matching.shape).encode("utf-8")+b"7b23b7829544f926").hexdigest() == "97e5c5211243030d0185cb7fca156656cf08d977", "order of elements of not_matching.shape is not correct"

assert sha1(str(type(sum(not_matching.predicted))).encode("utf-8")+b"becec9bb9498c2d1").hexdigest() == "41d7c30a2e58ad445119e09d6b35c9ee6c96e4a2", "type of sum(not_matching.predicted) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(not_matching.predicted)).encode("utf-8")+b"becec9bb9498c2d1").hexdigest() == "833d3a4c5500f0f577cb20544641d666d9438cbc", "value of sum(not_matching.predicted) is not correct"

assert sha1(str(type(sum(not_matching.index))).encode("utf-8")+b"80bdf4bfeabede68").hexdigest() == "c76dfa63e3a2ed852773c3d24f16a88d4306708c", "type of sum(not_matching.index) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(not_matching.index)).encode("utf-8")+b"80bdf4bfeabede68").hexdigest() == "fe0115f6e23b9da900c35bfbc3a19cf1ad38dbca", "value of sum(not_matching.index) is not correct"

assert sha1(str(type(not_matching_1)).encode("utf-8")+b"046c118c0ce13791").hexdigest() == "2a9adcdcd69aefe2af688ecb3afec151d73300ab", "type of not_matching_1 is not correct"
assert sha1(str(not_matching_1).encode("utf-8")+b"046c118c0ce13791").hexdigest() == "7697bcce7aa530087191cb9026244f1402df6d37", "value of not_matching_1 is not correct"

assert sha1(str(type(not_matching_2)).encode("utf-8")+b"e0de32a8846906f0").hexdigest() == "50d2639fdce62739565efa24ab060a2e4046ffd7", "type of not_matching_2 is not correct"
assert sha1(str(not_matching_2).encode("utf-8")+b"e0de32a8846906f0").hexdigest() == "8257abadd479e9af417d37a944969bde037ce757", "value of not_matching_2 is not correct"

assert sha1(str(type(not_matching_3)).encode("utf-8")+b"a948be714557fa85").hexdigest() == "ad518b0ed7989076371175472da98bec2252e626", "type of not_matching_3 is not correct"
assert sha1(str(not_matching_3).encode("utf-8")+b"a948be714557fa85").hexdigest() == "a33032e302b3f79c2723d5c4f7e312fa2191bb84", "value of not_matching_3 is not correct"

print('Success!')

**Question 4.4** True or False:
<br> {points: 1}

The above images were predicted incorrectly due to messy handwriting. For example, the second image is illegible and actually looks like the letter "y".

*Assign your answer to an object called `answer4_4`. Make sure your answer is a boolean (e.g. `True` or `False`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer4_4)).encode("utf-8")+b"0b057e7678538b86").hexdigest() == "ce6da045b267ac99fd7eec89c2e5a4890ebfef94", "type of answer4_4 is not bool. answer4_4 should be a bool"
assert sha1(str(answer4_4).encode("utf-8")+b"0b057e7678538b86").hexdigest() == "73d43273cf330e9c738bf4b03c6449810a3c39c3", "boolean value of answer4_4 is not correct"

print('Success!')

**Question 4.5**
<br> {points: 3}

Looking again at the plot from **Question 4.1**, what does this accuracy mean? Is it good enough that you would use this model for the Canada Post? Can you imagine a way we might improve our classifier's accuracy?

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.